## Conversion to fif: BJH030

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
## prep paths

subject = 'BJH030'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH030/PacmanTask/ECOG001/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [3]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [4]:
## check structure if necessary

my_states
# my_params

{'WebcamFrame0': array([[    0,     0,     0, ..., 23109, 23109, 23109]], dtype=int32),
 'WebcamFrame1': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'WebcamFrame2': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'WebcamFrame3': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'KeyDown': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'KeyUp': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'GhostLocation': array([[  0,   0,   0, ..., 132, 132, 132]], dtype=int32),
 'UserLocation': array([[ 0,  0,  0, ..., 57, 57, 57]], dtype=int32),
 'Direction': array([[4, 4, 4, ..., 2, 2, 2]], dtype=int32),
 'Biscuit1': array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32),
 'Biscuit2': array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32),
 'Biscuit3': array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32),
 'Biscuit4': array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32),
 'Biscuit5': array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32),
 'Attack': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Chase': array([[0, 0, 0, ..., 0, 0, 

In [5]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: AR1, AR2, AR3, AR4, REF1, REF2, AR5, AR6, BR1, BR2, BR3, BR4, ...
 chs: 256 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 256
 projs: []
 sfreq: 2000.0 Hz
>

In [6]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=256, n_times=3081800
    Range : 0 ... 3081799 =      0.000 ...  1540.899 secs
Ready.


In [7]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=3081800
    Range : 0 ... 3081799 =      0.000 ...  1540.899 secs
Ready.


<RawArray | 257 x 3081800 (1540.9 s), ~5.90 GB, data loaded>

In [8]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/BJH030/ieeg/BJH030_raw_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH030/ieeg/BJH030_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH030/ieeg/BJH030_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH030/ieeg/BJH030_raw_ieeg.fif
[done]
